In [4]:
import os
import shutil
import random

# Caminhos das pastas
train_dir = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\train'
test_dir = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\test'
valid_dir = r'C:\Mestrado\Materias\2024-02-Sistemas para Processamento Multimidia\seminario\image-segmentation\dataset\global-wheat-detection\val'

# Listar todos os arquivos .jpg na pasta train
files = [f for f in os.listdir(train_dir) if f.endswith('.jpg')]

# Embaralhar a lista de arquivos
random.shuffle(files)

# Calcular os índices para divisão
total_files = len(files)
train_count = int(total_files * 0.6)
test_count = int(total_files * 0.3)
valid_count = total_files - train_count - test_count

# Função para mover arquivos com rollback em caso de erro
def move_files_with_rollback(file_list, source_dir, dest_dir):
    moved_files = []
    try:
        for file in file_list:
            base_name = os.path.splitext(file)[0]
            jpg_file = os.path.join(source_dir, file)
            json_file = os.path.join(source_dir, base_name + '.json')
            
            shutil.move(jpg_file, dest_dir)
            moved_files.append((jpg_file, os.path.join(dest_dir, file)))
            
            if os.path.exists(json_file):
                shutil.move(json_file, dest_dir)
                moved_files.append((json_file, os.path.join(dest_dir, base_name + '.json')))
    except Exception as e:
        # Rollback
        for src, dest in moved_files:
            shutil.move(dest, src)
        raise e

# Mover arquivos para as pastas correspondentes com rollback
try:
    move_files_with_rollback(files[train_count:train_count + test_count], train_dir, test_dir)
    move_files_with_rollback(files[train_count + test_count:], train_dir, valid_dir)
except Exception as e:
    print(f"Erro ao mover arquivos: {e}")